# Part 3. Enhancement
The RNN model used in Part 2 is a basic model to perform the task of sentiment classification. In
this section, you will design strategies to improve upon the previous model you have built. You are
required to implement the following adjustments:

1. Instead of keeping the word embeddings fixed, now update the word embeddings (the same
way as model parameters) during the training process.
2. As discussed in Question 1(c), apply your solution in mitigating the influence of OOV words
and train your model again.
3. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a biLSTM model and a biGRU model, incorporating recurrent computations in both directions and
stacking multiple layers if possible.
4. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a Convolutional Neural Network (CNN) to produce sentence representations and perform sentiment
classification.
5. Further improve your model. You are free to use any strategy other than the above mentioned solutions. Changing hyper-parameters or stacking more layers is not counted towards
a meaningful improvement.


## Question 1

Instead of keeping the word embeddings fixed, now update the word embeddings (the same
way as model parameters) during the training process.

### Approach

We will use the same model as in part 2 notebook, but now we will also back propagate
the loss into the word embeddings itself. This will mean that as the model learns,
the word embeddings would also update, causing the encoding of the words to change.

We keep all other variables constant such as not handling the OOV words and using
the same hyperparameters as in part 2 notebook.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from common_utils import EmbeddingMatrix

In [ ]:
from common_utils import CustomDatasetPreparer, BATCH_SIZE


dataset_preparer = CustomDatasetPreparer(
    dataset_name="rotten_tomatoes",
    batch_size=BATCH_SIZE,
)

train_loader, val_loader, test_loader = dataset_preparer.get_dataloaders(ignore_unknown=True)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x,_,lenghts, sample_y = next(dataiter)

# print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)
print()
print('Sample label size: ', lenghts.size()) # batch_size
print('Sample label: \n', lenghts)

In [ ]:
class RNN(nn.Module):

    def __init__(
        self,
        hidden_dim: int,
        embedding_dim: int,
        word_embeddings: torch.Tensor,
        pad_idx,
        num_layers=1,
        output_size=1,
        dropout_rate=0
    ):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        self.embedding = nn.Embedding.from_pretrained(word_embeddings, freeze=False, padding_idx=pad_idx)
        self.rnn = nn.RNN(
            embedding_dim, hidden_dim, num_layers, batch_first=True
        )  # this is the num rows of the input matrix
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, text_lengths):
        embedded = self.embedding(x)
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        if self.dropout_rate > 0:
            embedded = self.dropout(self.embedding(x)).float()
        else:
            embedded = self.embedding(x).float()

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, batch_first=True, enforce_sorted = False)
        packed_output, _ = self.rnn(packed_embedded)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        
        if self.dropout_rate > 0:
            output = self.dropout(output)
        
        output, _ = torch.max(output, 1)

        out = self.fc(output)  # Use the last output of the RNN for classification

        sig_out = self.sigmoid(out)
        sig_out = sig_out.squeeze(1)
        # print("Sig_out shape: ", sig_out.shape)
        return sig_out

from common_utils import HIDDEN_SIZE, EMBEDDING_DIM, LEARNING_RATE
# initialize word embeddings
word_embeddings = EmbeddingMatrix.load()
word_embeddings.add_padding()

print("The index of <PAD> is: ", word_embeddings.pad_idx)


basic_RNN = RNN(
    hidden_dim=HIDDEN_SIZE,
    embedding_dim=EMBEDDING_DIM,
    word_embeddings=word_embeddings.to_tensor,
    dropout_rate=0.3,
    pad_idx=word_embeddings.pad_idx,
    num_layers=1,
)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 100
es_patience = 15

model = basic_RNN
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.BCELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.05)

model = model.to(device)
loss_fn = loss_fn.to(device)

In [ ]:
# function to predict accuracy
def acc(pred,label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label).item()

# training
def train_loop(train_loader, model, loss_fn, optimizer, scheduler, max_norm = 5, device='cpu'):
    train_loss = []
    train_acc = 0.0
    model.train()
    for X, extra_features, lengths, Y in train_loader:
        X, Y = X.to(device), Y.to(device)   

        optimizer.zero_grad()
        output = model(X, lengths)
        
        # calculate the loss and perform backprop
        loss = loss_fn(output.squeeze(), Y.float())
        train_loss.append(loss.item())
        loss.backward()
        
        # calculating accuracy
        accuracy = acc(output,Y)
        train_acc += accuracy
        
        #`clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        optimizer.step()
        
    scheduler.step()
    epoch_train_loss = np.mean(train_loss)
    epoch_train_acc = train_acc/len(train_loader.dataset)

    return epoch_train_loss, epoch_train_acc


def test_loop(test_loader, model, loss_fn, optimizer, device='cpu'):
    test_loss = []
    test_acc = 0.0
    model.eval()
    with torch.no_grad():
        for X,extra_features, lengths, Y in test_loader:
            X, Y = X.to(device), Y.to(device)   

            optimizer.zero_grad()
            output = model(X, lengths)
            
            # calculate the loss and perform backprop
            loss = loss_fn(output.squeeze(), Y.float())

            test_loss.append(loss.item())
            # calculating accuracy
            accuracy = acc(output,Y)
            test_acc += accuracy

    epoch_test_loss = np.mean(test_loss)
    epoch_test_acc = test_acc/len(test_loader.dataset)

    return epoch_test_loss, epoch_test_acc

def train_model(train_loader, val_loader, model, loss_fn, optimizer, scheduler, epochs, es_patience):
    best_val_loss = np.inf
    best_acc = 0
    train_loss_, train_acc_, val_loss_, val_acc_ = [], [], [], []
    from tqdm import tqdm
    # start training
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_loop(train_loader, model, loss_fn, optimizer, scheduler)
        val_loss, val_acc = test_loop(val_loader, model, loss_fn, optimizer)

        train_loss_.append(train_loss), train_acc_.append(train_acc)
        val_loss_.append(val_loss), val_acc_.append(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
        # early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            best_model = model.state_dict()
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= es_patience:
                print(f'early stopping after {epoch+1} epochs')
                print(f'best val loss: {best_val_loss}')
                print(f'best accuracy on val set: {best_acc}')
                break

        if epoch % 10 == 0:
            print(f"epoch {epoch+1}, train_loss {train_loss:>7f} train_acc {train_acc:>4f}, val_loss {val_loss:>7f}, val_acc {val_acc:>4f}")

    return train_loss_, train_acc_, val_loss_, val_acc_

train_loss_, train_acc_, val_loss_, val_acc_ = train_model(train_loader, val_loader, model, loss_fn, optimizer, scheduler, epochs, es_patience)

# Question 3

(a) We use the same RNN model with the same parameters and we found that by unfreezing the weights, the performance of the model went for the test set. We hypothesize this is the case as the word embeddings are
generalized and not specific to the task at hand. By updating the word embeddings, the model can learn the specific embeddings for the small dataset that was given to us.

We show our results in the following 2 cells where we obtained a much better accuracy on the validation and test set.

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_accuracy(train_loss_, train_acc_, val_loss_, val_acc_):
    fig = plt.figure(figsize = (20, 6))
    plt.subplot(1, 2, 1)
    plt.plot(train_acc_, label='Train Acc')
    plt.plot(val_acc_, label='Validation Acc')
    plt.title("Accuracy")
    plt.legend()
    plt.grid()
        
    plt.subplot(1, 2, 2)
    plt.plot(train_loss_, label='Train loss')
    plt.plot(val_loss_, label='Validation loss')
    plt.title("Loss")
    plt.legend()
    plt.grid()

    plt.show()

plot_loss_accuracy(train_loss_, train_acc_, val_loss_, val_acc_)

In [ ]:
test_loss, test_acc = test_loop(test_loader, model, loss_fn)
print(f"test_loss {test_loss:>7f}, test_acc {test_acc:>4f}")

## Question 2

### Approach

As discussed in part 1, we have mentioned 2 approaches to handling of the
OOV words. We will now demonstrate the first approach, which is to replace the OOV
words with a special token. We will replace the OOV words with a special token
`<UNK>`.

In [ ]:
from common_utils import EMBEDDING_DIM, UNK_TOKEN, load_glove_embeddings


w2v_model = EmbeddingMatrix.load()
extended_vocab = w2v_model.vocab
extended_vocab.add(UNK_TOKEN)

glove_dict = load_glove_embeddings()

# Collect words to be removed
missing_words = []
for word in extended_vocab:
    if word not in glove_dict:
        missing_words.append(word)

# Remove missing words from vocab
for word in missing_words:
    extended_vocab.remove(word)
        
print(f"Number of missing words: {len(missing_words)}")
print(f"The missing words are: {missing_words}")

# mapping of words to indices and vice versa
word2idx = {word: idx for idx, word in enumerate(sorted(extended_vocab))}
idx2word = {idx: word for word, idx in word2idx.items()}

print("Building embedding matrix...")
vocab_size = len(word2idx)
print(f"Vocab size: {vocab_size}")
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, idx in word2idx.items():
    embedding_matrix[idx] = glove_dict[word]

# add random vector for unknown words
embedding_matrix[word2idx[UNK_TOKEN]] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))

print("Embedding matrix built successfully.")


To aid us in the downstream task for the rest of the analysis that we will be
doing, we will update the EmbeddingMatrix class that was earlier defined to
also handle the case of unknown words.

We define the class using an API interface which can be easily called.

In [ ]:
from common_utils import EMBEDDING_MATRIX_PATH, WORD2IDX_PATH, IDX2WORD_PATH

class EmbeddingMatrix:
    def __init__(self, unk_token=UNK_TOKEN, handle_unknown=True) -> None:
        self.d = 0
        self.v = 0
        self.pad_idx: int
        self.unk_idx: int
        self.embedding_matrix: np.ndarray
        self.word2idx: dict
        self.idx2word: dict
        self.unk_token = unk_token
        self.handle_unknown = handle_unknown

    @classmethod
    def load(cls) -> "EmbeddingMatrix":
        # load vectors from file
        embedding_matrix: np.ndarray = np.load(EMBEDDING_MATRIX_PATH)
        # set attributes
        em = cls()
        em.embedding_matrix = embedding_matrix

        with open(WORD2IDX_PATH, "r", encoding="utf-8") as f:
            word2idx: dict = json.load(f)
            em.word2idx = word2idx
        with open(IDX2WORD_PATH, "r", encoding="utf-8") as f:
            idx2word: dict = json.load(f)
            em.idx2word = idx2word
            
        em.v, em.d = embedding_matrix.shape
        return em

    def save(self) -> None:
        np.save(EMBEDDING_MATRIX_PATH, self.embedding_matrix)
        
        with open(WORD2IDX_PATH, "w", encoding="utf-8") as f:
            json.dump(self.word2idx, f, ensure_ascii=False, indent=4)
            
        with open(IDX2WORD_PATH, "w", encoding="utf-8") as f:
            json.dump(self.idx2word, f, ensure_ascii=False, indent=4)

    @property
    def to_tensor(self) -> torch.Tensor:
        return torch.tensor(self.embedding_matrix, dtype=torch.float)

    def add_padding(self) -> None:
        if "<PAD>" in self.word2idx:
            return
        padding = np.zeros((1, self.d), dtype="float32")
        self.embedding_matrix = np.vstack((self.embedding_matrix, padding))

        self.v += 1
        self.pad_idx = self.v - 1
        self.word2idx["<PAD>"] = self.pad_idx

    def add_unk_token(self) -> None:
        if self.unk_token in self.word2idx:
            return
        unk_vector = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))
        self.embedding_matrix = np.vstack((self.embedding_matrix, unk_vector))

        self.v += 1
        self.unk_idx = self.v - 1
        self.word2idx[self.unk_token] = self.unk_idx

    @property
    def dimension(self) -> int:
        """Dimension of the embedding matrix

        :return: The dimension of the embedding matrix
        :rtype: int
        """
        return self.d

    @property
    def vocab_size(self) -> int:
        """Vocabulary size of the embedding matrix

        :return: The vocabulary size of the embedding matrix
        :rtype: int
        """
        return self.v

    @property
    def vocab(self) -> set[str]:
        """Vocabulary of the embedding matrix

        Set of words in the embedding matrix

        :return: The vocabulary of the embedding matrix
        :rtype: set[str]
        """
        return set(self.word2idx.keys())

    def __getitem__(self, word: str) -> np.ndarray:
        return self.embedding_matrix[self.word2idx[word]]

    def get_idx(self, word: str) -> int:
        # if word not in vocab, return None
        if self.handle_unknown:
            return self.word2idx.get(word, self.unk_idx)

        return self.word2idx.get(word, None)
    
    def is_in_vocab(self, word: str) -> bool:
        return word in self.word2idx

    def is_in_index(self, idx: int) -> bool:
        return idx in self.idx2word

For the second approach, we will use the FastText embeddings, which are trained on
subword information. This will help in encoding the OOV words as well.

In [ ]:
# Implementation of FastText for word embedding
import gensim
from gensim.models import FastText
from common_utils import EMBEDDING_DIM
# Create a FastText model with the same dimensions as the Word2Vec model
fasttext_model = FastText(
    vector_size=EMBEDDING_DIM,
    window=5, # context window size 
    min_count=1, # threshold for word frequency
    workers=4
)

In [ ]:
from datasets import load_dataset, Dataset
from common_utils import EmbeddingMatrix, tokenize
import nltk
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']

corpus = []
for example in train_dataset:
    tokens = nltk.word_tokenize(example['text'])
    corpus.append(tokens)
print("The corpus has {} documents.".format(len(corpus)))

In [ ]:
fasttext_model.build_vocab(corpus_iterable=corpus)
# fasttext_model.build_vocab(corpus_iterable=corpus, update=True)

print("Length of vocabulary:", len(fasttext_model.wv.key_to_index))


fasttext_model.train(
    corpus_iterable=corpus, epochs=fasttext_model.epochs,
    total_examples=fasttext_model.corpus_count, total_words=fasttext_model.corpus_total_words,
)

In [ ]:
"computer" in fasttext_model.wv.key_to_index

In [ ]:
fasttext_vocab = set(fasttext_model.wv.key_to_index.keys())
w2v_model = EmbeddingMatrix.load()

In [ ]:
w2v_model.vocab - fasttext_vocab

In [ ]:
fasttext_vocab - w2v_model.vocab

In [ ]:
print("The FastText model has {} words.".format(len(fasttext_model.wv.key_to_index)))
print("The Word2Vec model has {} words.".format(len(w2v_model.vocab)))
print("The FastText model has {} words that are not in the Word2Vec model.".format(len(fasttext_vocab - w2v_model.vocab)))
print("The Word2Vec model has {} words that are not in the FastText model.".format(len(w2v_model.vocab - fasttext_vocab)))

In [ ]:
# save the FastText model
fasttext_model.save("fasttext_model.model")

With the OOV handling defined, we now define the handing of the dataloader
with the new embedding containing the `<UNK>` token. We will also now
define an RNN model with the updated embedding matrix and run it on the
data loader to test the performance gain.

In [ ]:
from common_utils import CustomDatasetPreparer, BATCH_SIZE


dataset_preparer = CustomDatasetPreparer(
    dataset_name="rotten_tomatoes",
    batch_size=BATCH_SIZE,
)

train_loader, val_loader, test_loader = dataset_preparer.get_dataloaders()

In [ ]:
# initialize word embeddings
word_embeddings = EmbeddingMatrix.load()
word_embeddings.add_padding()
word_embeddings.add_unk_token()

print("The index of <PAD> is: ", word_embeddings.pad_idx)


basic_RNN = RNN(
    hidden_dim=HIDDEN_SIZE,
    embedding_dim=EMBEDDING_DIM,
    word_embeddings=word_embeddings.to_tensor,
    dropout_rate=0.3,
    pad_idx=word_embeddings.pad_idx,
    num_layers=1,
)


# Question 3

(b) The following code shows the performance of the model with OOV handling
where the model would have the learn the embeddings for the unknown words
that it might occur and subsequently how to classify an `<UNK>` token that.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 100
es_patience = 15

model = basic_RNN
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.BCELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.05)

model = model.to(device)
loss_fn = loss_fn.to(device)

train_loss_, train_acc_, val_loss_, val_acc_ = train_model(train_loader, val_loader, model, loss_fn, optimizer, scheduler, epochs, es_patience)


In [ ]:
plot_loss_accuracy(train_loss_, train_acc_, val_loss_, val_acc_)

test_loss, test_acc = test_loop(test_loader, model, loss_fn, optimizer)
print(f"test_loss {test_loss:>7f}, test_acc {test_acc:>4f}")